In [1]:
system("rm -r Results")

In [2]:
system("mkdir Results")

In [3]:
system("gsutil cp gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/all_phenotypes.csv Results/")
for(trait in c("BMI","LDL","HDL","logTG","TC","Height")){
    system(paste0("gsutil cp gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/RareVariant_PRS/",trait,"_PRS_Validation.csv Results/"))
    system(paste0("gsutil cp gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/OneCommonPRS/",trait,"_Best_Validation_All.txt Results/"))
}

In [4]:
system("gsutil cp gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Validation.txt Results/")

In [5]:
list.files("Results/")

[1] "all_phenotypes.csv"             "All_Validation.txt"            
 [3] "BMI_Best_Validation_All.txt"    "BMI_PRS_Validation.csv"        
 [5] "HDL_Best_Validation_All.txt"    "HDL_PRS_Validation.csv"        
 [7] "Height_Best_Validation_All.txt" "Height_PRS_Validation.csv"     
 [9] "LDL_Best_Validation_All.txt"    "LDL_PRS_Validation.csv"        
[11] "logTG_Best_Validation_All.txt"  "logTG_PRS_Validation.csv"      
[13] "TC_Best_Validation_All.txt"     "TC_PRS_Validation.csv"

In [6]:
install.packages("ggthemes")

Installing package into ‘/home/jupyter/packages’
(as ‘lib’ is unspecified)



In [11]:
trait <- "TC"
library(dplyr)
  library(ggplot2)
  library(cowplot)
  
  pheno_validation <- read.delim("Results/All_Validation.txt")
  CV_PRS_Validation <- read.delim(paste0("Results/",trait,"_Best_Validation_All.txt"))
  colnames(CV_PRS_Validation) <- c("IID","CV_PRS")
  pheno_validation <- inner_join(pheno_validation,CV_PRS_Validation)
  RV_PRS_Validation <- read.csv(paste0("Results/",trait,"_PRS_Validation.csv"))
  colnames(RV_PRS_Validation) <- c("IID","RV_PRS")
  pheno_validation <- inner_join(pheno_validation,RV_PRS_Validation)
  
  model.null <- lm(as.formula(paste0(trait,"~age+age2+sex+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10")),data=pheno_validation)
  pheno_validation$y_validation <- NA
  pheno_validation$y_validation[!is.na(pheno_validation[,trait])] <- model.null$residual
  
  CV_RV_PRS_raw <- pheno_validation
  CV_RV_PRS_adjusted <- pheno_validation
  
  for(i in c("RV_PRS","CV_PRS")){
    tmp <- data.frame(y = CV_RV_PRS_adjusted[,i],CV_RV_PRS_adjusted[,c("PC1","PC2","PC3","PC4","PC5")])
    mod <- lm(y~.,data = tmp)
    R <- mod$residuals
    tmp <- data.frame(y = R^2,CV_RV_PRS_adjusted[,c("PC1","PC2","PC3","PC4","PC5")])
    mod <- lm(y~.,data = tmp)
    y_hat <- predict(mod,tmp)
    if(sum(y_hat < 0) > 0){
      mod <- lm(y~1,data = tmp)
      y_hat <- predict(mod,tmp)
    }
    if(sum(sqrt(y_hat)) == 0){
      CV_RV_PRS_adjusted[,i] <- 0
    }else{
      CV_RV_PRS_adjusted[,i] <- R/sqrt(y_hat)
    }
  }
  
  
  CV_RV_PRS_adjusted$Common_Bin <- 9
  CV_RV_PRS_adjusted$Rare_Bin <- 9
  
  Common_quants <- quantile(CV_RV_PRS_adjusted$CV_PRS,c(0,.1,.2,.3,.4,.6,.7,.8,.9))
  
  for(i in 1:8){
    CV_RV_PRS_adjusted$Common_Bin[CV_RV_PRS_adjusted$CV_PRS < unname(Common_quants[i + 1]) & CV_RV_PRS_adjusted$CV_PRS >= unname(Common_quants[i])] <- i
  }
  
  ukb_pheno <- read.csv("Results/all_phenotypes.csv")
  
  CV_RV_PRS_adjusted_EUR <- CV_RV_PRS_adjusted[CV_RV_PRS_adjusted$IID %in% ukb_pheno$IID[ukb_pheno$ancestry == "EUR"],]
  CV_RV_PRS_adjusted_AMR <- CV_RV_PRS_adjusted[CV_RV_PRS_adjusted$IID %in% ukb_pheno$IID[ukb_pheno$ancestry == "AMR"],]
  CV_RV_PRS_adjusted_AFR <- CV_RV_PRS_adjusted[CV_RV_PRS_adjusted$IID %in% ukb_pheno$IID[ukb_pheno$ancestry == "AFR"],]
  CV_RV_PRS_adjusted_SAS <- CV_RV_PRS_adjusted[CV_RV_PRS_adjusted$IID %in% ukb_pheno$IID[ukb_pheno$ancestry == "SAS"],]
  CV_RV_PRS_adjusted_EAS <- CV_RV_PRS_adjusted[CV_RV_PRS_adjusted$IID %in% ukb_pheno$IID[ukb_pheno$ancestry == "EAS"],]
  CV_RV_PRS_adjusted_MID <- CV_RV_PRS_adjusted[CV_RV_PRS_adjusted$IID %in% ukb_pheno$IID[ukb_pheno$ancestry == "MID"],]
  
  Rare_quants <- quantile(CV_RV_PRS_adjusted_EUR$RV_PRS,c(0,.05,.2,.3,.4,.6,.7,.8,.95))
  for(i in 1:8){
    CV_RV_PRS_adjusted_EUR$Rare_Bin[CV_RV_PRS_adjusted_EUR$RV_PRS < unname(Rare_quants[i + 1]) & CV_RV_PRS_adjusted_EUR$RV_PRS >= unname(Rare_quants[i])] <- i
  }
  
  CV_RV_PRS_adjusted_EUR$Rare_Bin[CV_RV_PRS_adjusted_EUR$Rare_Bin == 1] <- "Below 5%"
  CV_RV_PRS_adjusted_EUR$Rare_Bin[CV_RV_PRS_adjusted_EUR$Rare_Bin %in% c("4","5","6")] <- "30% - 70%"
  CV_RV_PRS_adjusted_EUR$Rare_Bin[CV_RV_PRS_adjusted_EUR$Rare_Bin %in% c("9")] <- "Above 95%"
  
  CV_RV_PRS_adjusted_EUR <- CV_RV_PRS_adjusted_EUR[CV_RV_PRS_adjusted_EUR$Rare_Bin %in% c("Below 5%","30% - 70%","Above 95%"),]
  
  CV_RV_PRS_adjusted_EUR$Rare_Bin <- factor(CV_RV_PRS_adjusted_EUR$Rare_Bin,levels = c("Below 5%","30% - 70%","Above 95%"))
  
  Rare_quants <- quantile(CV_RV_PRS_adjusted_AMR$RV_PRS,c(0,.05,.2,.3,.4,.6,.7,.8,.95))
  for(i in 1:8){
    CV_RV_PRS_adjusted_AMR$Rare_Bin[CV_RV_PRS_adjusted_AMR$RV_PRS < unname(Rare_quants[i + 1]) & CV_RV_PRS_adjusted_AMR$RV_PRS >= unname(Rare_quants[i])] <- i
  }
  
  CV_RV_PRS_adjusted_AMR$Rare_Bin[CV_RV_PRS_adjusted_AMR$Rare_Bin == 1] <- "Below 5%"
  CV_RV_PRS_adjusted_AMR$Rare_Bin[CV_RV_PRS_adjusted_AMR$Rare_Bin %in% c("4","5","6")] <- "30% - 70%"
  CV_RV_PRS_adjusted_AMR$Rare_Bin[CV_RV_PRS_adjusted_AMR$Rare_Bin %in% c("9")] <- "Above 95%"
  
  CV_RV_PRS_adjusted_AMR <- CV_RV_PRS_adjusted_AMR[CV_RV_PRS_adjusted_AMR$Rare_Bin %in% c("Below 5%","30% - 70%","Above 95%"),]
  
  CV_RV_PRS_adjusted_AMR$Rare_Bin <- factor(CV_RV_PRS_adjusted_AMR$Rare_Bin,levels = c("Below 5%","30% - 70%","Above 95%"))
  
  Rare_quants <- quantile(CV_RV_PRS_adjusted_AFR$RV_PRS,c(0,.05,.2,.3,.4,.6,.7,.8,.95))
  for(i in 1:8){
    CV_RV_PRS_adjusted_AFR$Rare_Bin[CV_RV_PRS_adjusted_AFR$RV_PRS < unname(Rare_quants[i + 1]) & CV_RV_PRS_adjusted_AFR$RV_PRS >= unname(Rare_quants[i])] <- i
  }
  
  CV_RV_PRS_adjusted_AFR$Rare_Bin[CV_RV_PRS_adjusted_AFR$Rare_Bin == 1] <- "Below 5%"
  CV_RV_PRS_adjusted_AFR$Rare_Bin[CV_RV_PRS_adjusted_AFR$Rare_Bin %in% c("4","5","6")] <- "30% - 70%"
  CV_RV_PRS_adjusted_AFR$Rare_Bin[CV_RV_PRS_adjusted_AFR$Rare_Bin %in% c("9")] <- "Above 95%"
  
  CV_RV_PRS_adjusted_AFR <- CV_RV_PRS_adjusted_AFR[CV_RV_PRS_adjusted_AFR$Rare_Bin %in% c("Below 5%","30% - 70%","Above 95%"),]
  
  CV_RV_PRS_adjusted_AFR$Rare_Bin <- factor(CV_RV_PRS_adjusted_AFR$Rare_Bin,levels = c("Below 5%","30% - 70%","Above 95%"))
  
  Rare_quants <- quantile(CV_RV_PRS_adjusted_SAS$RV_PRS,c(0,.05,.2,.3,.4,.6,.7,.8,.95))
  for(i in 1:8){
    CV_RV_PRS_adjusted_SAS$Rare_Bin[CV_RV_PRS_adjusted_SAS$RV_PRS < unname(Rare_quants[i + 1]) & CV_RV_PRS_adjusted_SAS$RV_PRS >= unname(Rare_quants[i])] <- i
  }
  
  CV_RV_PRS_adjusted_SAS$Rare_Bin[CV_RV_PRS_adjusted_SAS$Rare_Bin == 1] <- "Below 5%"
  CV_RV_PRS_adjusted_SAS$Rare_Bin[CV_RV_PRS_adjusted_SAS$Rare_Bin %in% c("4","5","6")] <- "30% - 70%"
  CV_RV_PRS_adjusted_SAS$Rare_Bin[CV_RV_PRS_adjusted_SAS$Rare_Bin %in% c("9")] <- "Above 95%"
  
  CV_RV_PRS_adjusted_SAS <- CV_RV_PRS_adjusted_SAS[CV_RV_PRS_adjusted_SAS$Rare_Bin %in% c("Below 5%","30% - 70%","Above 95%"),]
  
  CV_RV_PRS_adjusted_SAS$Rare_Bin <- factor(CV_RV_PRS_adjusted_SAS$Rare_Bin,levels = c("Below 5%","30% - 70%","Above 95%"))
  
  
  Rare_quants <- quantile(CV_RV_PRS_adjusted_EAS$RV_PRS,c(0,.05,.2,.3,.4,.6,.7,.8,.95))
  for(i in 1:8){
    CV_RV_PRS_adjusted_EAS$Rare_Bin[CV_RV_PRS_adjusted_EAS$RV_PRS < unname(Rare_quants[i + 1]) & CV_RV_PRS_adjusted_EAS$RV_PRS >= unname(Rare_quants[i])] <- i
  }
  
  CV_RV_PRS_adjusted_EAS$Rare_Bin[CV_RV_PRS_adjusted_EAS$Rare_Bin == 1] <- "Below 5%"
  CV_RV_PRS_adjusted_EAS$Rare_Bin[CV_RV_PRS_adjusted_EAS$Rare_Bin %in% c("4","5","6")] <- "30% - 70%"
  CV_RV_PRS_adjusted_EAS$Rare_Bin[CV_RV_PRS_adjusted_EAS$Rare_Bin %in% c("9")] <- "Above 95%"
  
  CV_RV_PRS_adjusted_EAS <- CV_RV_PRS_adjusted_EAS[CV_RV_PRS_adjusted_EAS$Rare_Bin %in% c("Below 5%","30% - 70%","Above 95%"),]
  
  CV_RV_PRS_adjusted_EAS$Rare_Bin <- factor(CV_RV_PRS_adjusted_EAS$Rare_Bin,levels = c("Below 5%","30% - 70%","Above 95%"))
  
  
  
  Rare_quants <- quantile(CV_RV_PRS_adjusted_MID$RV_PRS,c(0,.05,.2,.3,.4,.6,.7,.8,.95))
  for(i in 1:8){
    CV_RV_PRS_adjusted_MID$Rare_Bin[CV_RV_PRS_adjusted_MID$RV_PRS < unname(Rare_quants[i + 1]) & CV_RV_PRS_adjusted_MID$RV_PRS >= unname(Rare_quants[i])] <- i
  }
  
  CV_RV_PRS_adjusted_MID$Rare_Bin[CV_RV_PRS_adjusted_MID$Rare_Bin == 1] <- "Below 5%"
  CV_RV_PRS_adjusted_MID$Rare_Bin[CV_RV_PRS_adjusted_MID$Rare_Bin %in% c("4","5","6")] <- "30% - 70%"
  CV_RV_PRS_adjusted_MID$Rare_Bin[CV_RV_PRS_adjusted_MID$Rare_Bin %in% c("9")] <- "Above 95%"
  
  CV_RV_PRS_adjusted_MID <- CV_RV_PRS_adjusted_MID[CV_RV_PRS_adjusted_MID$Rare_Bin %in% c("Below 5%","30% - 70%","Above 95%"),]
  
  CV_RV_PRS_adjusted_MID$Rare_Bin <- factor(CV_RV_PRS_adjusted_MID$Rare_Bin,levels = c("Below 5%","30% - 70%","Above 95%"))
  
  
  
  CV_RV_PRS_adjusted_EUR$y_validation <- scale(CV_RV_PRS_adjusted_EUR$y_validation)
  CV_RV_PRS_adjusted_AMR$y_validation <- scale(CV_RV_PRS_adjusted_AMR$y_validation)
  CV_RV_PRS_adjusted_AFR$y_validation <- scale(CV_RV_PRS_adjusted_AFR$y_validation)
  CV_RV_PRS_adjusted_SAS$y_validation <- scale(CV_RV_PRS_adjusted_SAS$y_validation)
  CV_RV_PRS_adjusted_EAS$y_validation <- scale(CV_RV_PRS_adjusted_EAS$y_validation)
  CV_RV_PRS_adjusted_MID$y_validation <- scale(CV_RV_PRS_adjusted_MID$y_validation)
  
  CV_RV_PRS_adjusted_EUR_se <- aggregate(y_validation ~ Common_Bin + Rare_Bin,data = CV_RV_PRS_adjusted_EUR,function(x){sd(x)/sqrt(length(x))})
  CV_RV_PRS_adjusted_EUR <- aggregate(y_validation ~ Common_Bin + Rare_Bin,data = CV_RV_PRS_adjusted_EUR,mean)
  
  colnames(CV_RV_PRS_adjusted_EUR) <- c("Common_Bin","Rare_Bin","Mean")
  colnames(CV_RV_PRS_adjusted_EUR_se) <- c("Common_Bin","Rare_Bin","SE")
  CV_RV_PRS_adjusted_EUR <- inner_join(CV_RV_PRS_adjusted_EUR,CV_RV_PRS_adjusted_EUR_se)
  
  CV_RV_PRS_adjusted_AMR_se <- aggregate(y_validation ~ Common_Bin + Rare_Bin,data = CV_RV_PRS_adjusted_AMR,function(x){sd(x)/sqrt(length(x))})
  CV_RV_PRS_adjusted_AMR <- aggregate(y_validation ~ Common_Bin + Rare_Bin,data = CV_RV_PRS_adjusted_AMR,mean)
  
  colnames(CV_RV_PRS_adjusted_AMR) <- c("Common_Bin","Rare_Bin","Mean")
  colnames(CV_RV_PRS_adjusted_AMR_se) <- c("Common_Bin","Rare_Bin","SE")
  CV_RV_PRS_adjusted_AMR <- inner_join(CV_RV_PRS_adjusted_AMR,CV_RV_PRS_adjusted_AMR_se)
  
  CV_RV_PRS_adjusted_AFR_se <- aggregate(y_validation ~ Common_Bin + Rare_Bin,data = CV_RV_PRS_adjusted_AFR,function(x){sd(x)/sqrt(length(x))})
  CV_RV_PRS_adjusted_AFR <- aggregate(y_validation ~ Common_Bin + Rare_Bin,data = CV_RV_PRS_adjusted_AFR,mean)
  
  colnames(CV_RV_PRS_adjusted_AFR) <- c("Common_Bin","Rare_Bin","Mean")
  colnames(CV_RV_PRS_adjusted_AFR_se) <- c("Common_Bin","Rare_Bin","SE")
  CV_RV_PRS_adjusted_AFR <- inner_join(CV_RV_PRS_adjusted_AFR,CV_RV_PRS_adjusted_AFR_se)
  
  CV_RV_PRS_adjusted_SAS_se <- aggregate(y_validation ~ Common_Bin + Rare_Bin,data = CV_RV_PRS_adjusted_SAS,function(x){sd(x)/sqrt(length(x))})
  CV_RV_PRS_adjusted_SAS <- aggregate(y_validation ~ Common_Bin + Rare_Bin,data = CV_RV_PRS_adjusted_SAS,mean)
  
  colnames(CV_RV_PRS_adjusted_SAS) <- c("Common_Bin","Rare_Bin","Mean")
  colnames(CV_RV_PRS_adjusted_SAS_se) <- c("Common_Bin","Rare_Bin","SE")
  CV_RV_PRS_adjusted_SAS <- inner_join(CV_RV_PRS_adjusted_SAS,CV_RV_PRS_adjusted_SAS_se)
  
  CV_RV_PRS_adjusted_EAS_se <- aggregate(y_validation ~ Common_Bin + Rare_Bin,data = CV_RV_PRS_adjusted_EAS,function(x){sd(x)/sqrt(length(x))})
  CV_RV_PRS_adjusted_EAS <- aggregate(y_validation ~ Common_Bin + Rare_Bin,data = CV_RV_PRS_adjusted_EAS,mean)
  
  colnames(CV_RV_PRS_adjusted_EAS) <- c("Common_Bin","Rare_Bin","Mean")
  colnames(CV_RV_PRS_adjusted_EAS_se) <- c("Common_Bin","Rare_Bin","SE")
  CV_RV_PRS_adjusted_EAS <- inner_join(CV_RV_PRS_adjusted_EAS,CV_RV_PRS_adjusted_EAS_se)
  
  CV_RV_PRS_adjusted_MID_se <- aggregate(y_validation ~ Common_Bin + Rare_Bin,data = CV_RV_PRS_adjusted_MID,function(x){sd(x)/sqrt(length(x))})
  CV_RV_PRS_adjusted_MID <- aggregate(y_validation ~ Common_Bin + Rare_Bin,data = CV_RV_PRS_adjusted_MID,mean)
  
  colnames(CV_RV_PRS_adjusted_MID) <- c("Common_Bin","Rare_Bin","Mean")
  colnames(CV_RV_PRS_adjusted_MID_se) <- c("Common_Bin","Rare_Bin","SE")
  CV_RV_PRS_adjusted_MID <- inner_join(CV_RV_PRS_adjusted_MID,CV_RV_PRS_adjusted_MID_se)
  
  
  colnames(CV_RV_PRS_adjusted_EUR) <- c("Common_Bin","RICE-RV Quantiles (Rare Variants)","Mean","SE")
  colnames(CV_RV_PRS_adjusted_AMR) <- c("Common_Bin","RICE-RV Quantiles (Rare Variants)","Mean","SE")
  colnames(CV_RV_PRS_adjusted_AFR) <- c("Common_Bin","RICE-RV Quantiles (Rare Variants)","Mean","SE")
  colnames(CV_RV_PRS_adjusted_SAS) <- c("Common_Bin","RICE-RV Quantiles (Rare Variants)","Mean","SE")
  colnames(CV_RV_PRS_adjusted_EAS) <- c("Common_Bin","RICE-RV Quantiles (Rare Variants)","Mean","SE")
  colnames(CV_RV_PRS_adjusted_MID) <- c("Common_Bin","RICE-RV Quantiles (Rare Variants)","Mean","SE")
  
  ymin <- round(min(c(CV_RV_PRS_adjusted_EUR$Mean - CV_RV_PRS_adjusted_EUR$SE,CV_RV_PRS_adjusted_AMR$Mean - CV_RV_PRS_adjusted_AMR$SE,CV_RV_PRS_adjusted_AFR$Mean - CV_RV_PRS_adjusted_AFR$SE,CV_RV_PRS_adjusted_SAS$Mean - CV_RV_PRS_adjusted_SAS$SE,CV_RV_PRS_adjusted_EAS$Mean - CV_RV_PRS_adjusted_EAS$SE,CV_RV_PRS_adjusted_MID$Mean - CV_RV_PRS_adjusted_MID$SE)) - 0.05,2)
  ymax <- round(max(c(CV_RV_PRS_adjusted_EUR$Mean + CV_RV_PRS_adjusted_EUR$SE,CV_RV_PRS_adjusted_AMR$Mean + CV_RV_PRS_adjusted_AMR$SE,CV_RV_PRS_adjusted_AFR$Mean + CV_RV_PRS_adjusted_AFR$SE,CV_RV_PRS_adjusted_SAS$Mean + CV_RV_PRS_adjusted_SAS$SE,CV_RV_PRS_adjusted_EAS$Mean + CV_RV_PRS_adjusted_EAS$SE,CV_RV_PRS_adjusted_MID$Mean + CV_RV_PRS_adjusted_MID$SE)) + 0.05,2)
  

Joining with `by = join_by(IID)`
Joining with `by = join_by(IID)`
Joining with `by = join_by(Common_Bin, Rare_Bin)`
Joining with `by = join_by(Common_Bin, Rare_Bin)`
Joining with `by = join_by(Common_Bin, Rare_Bin)`
Joining with `by = join_by(Common_Bin, Rare_Bin)`
Joining with `by = join_by(Common_Bin, Rare_Bin)`
Joining with `by = join_by(Common_Bin, Rare_Bin)`


In [12]:
aggregate(Mean ~ `RICE-RV Quantiles (Rare Variants)`, data = CV_RV_PRS_adjusted_EUR,mean)
aggregate(Mean ~ `RICE-RV Quantiles (Rare Variants)`, data = CV_RV_PRS_adjusted_AMR,mean)
aggregate(Mean ~ `RICE-RV Quantiles (Rare Variants)`, data = CV_RV_PRS_adjusted_AFR,mean)

RICE-RV Quantiles (Rare Variants),Mean
<fct>,<dbl>
Below 5%,-0.087545858
30% - 70%,0.003425977
Above 95%,0.060572827


RICE-RV Quantiles (Rare Variants),Mean
<fct>,<dbl>
Below 5%,-0.0434375943
30% - 70%,-0.0003948956
Above 95%,0.0328178187


RICE-RV Quantiles (Rare Variants),Mean
<fct>,<dbl>
Below 5%,-0.31001934
30% - 70%,0.02922573
Above 95%,0.10193708


In [13]:
0.10193708 - 0.02922573

[1] 0.07271135

In [14]:
0.02922573 - -0.31001934


[1] 0.3392451

In [24]:
rm(list = ls())

library(dplyr)
library(ggplot2)
library(cowplot)

theme_Publication <- function(base_size=12) {
  library(grid)
  library(ggthemes)
  (theme_foundation(base_size=base_size, )
    + theme(plot.title = element_text(face = "bold",
                                      size = rel(1.1), hjust = 0.5),
            text = element_text(),
            panel.background = element_rect(colour = NA),
            plot.background = element_rect(colour = NA),
            panel.border = element_rect(colour = NA),
            axis.title = element_text(face = "bold",size = 24),
            strip.text.x = element_text(face = "bold",size = 24),
            axis.title.y = element_text(angle=90,vjust =2),
            axis.line = element_line(colour="black",size=2),
            axis.text.y = element_text(size = 20),
            axis.text.x = element_text(size = 13),
            axis.ticks = element_line(),
            # panel.grid.major = element_line(colour="#f0f0f0"),
            # panel.grid.minor = element_line(colour="#f0f0f0"),
            panel.grid.major = element_blank(),
            panel.grid.minor = element_blank(),
            legend.key = element_rect(colour = NA),
            legend.position = "bottom",
            legend.text=element_text(size=20),
            #legend.direction = "horizontal",
            #legend.key.size= unit(0.2, "cm"),
            #legend.margin = unit(0, "cm"),
            legend.title = element_text(face="bold.italic", size =24),
            #legend.text = element_text(face ="bold"),
            plot.margin=unit(c(10,5,5,5),"mm"),
            strip.background=element_rect(colour="#f0f0f0",fill="#f0f0f0"),
            strip.text = element_text(face="bold")
    ))
  
}


scale_fill_Publication <- function(...){
  library(scales)
  discrete_scale("fill","Publication",manual_pal(values = c("#386cb0","#EF7E3D","#ffd558","#7fc97f","#ef3b2c","#662506","#a6cee3","#fb9a99","#984ea3","#ffff33")), ...)
  
}

scale_colour_Publication <- function(...){
  library(scales)
  discrete_scale("colour","Publication",manual_pal(values = c("#386cb0","#EF7E3D","#ffd558","#7fc97f","#ef3b2c","#662506","#a6cee3","#fb9a99","#984ea3","#ffff33")), ...)
  
}

for(trait in c("BMI","HDL","LDL","logTG","TC","Height")){
  library(dplyr)
  library(ggplot2)
  library(cowplot)
  
  pheno_validation <- read.delim("Results/All_Validation.txt")
  CV_PRS_Validation <- read.delim(paste0("Results/",trait,"_Best_Validation_All.txt"))
  colnames(CV_PRS_Validation) <- c("IID","CV_PRS")
  pheno_validation <- inner_join(pheno_validation,CV_PRS_Validation)
  RV_PRS_Validation <- read.csv(paste0("Results/",trait,"_PRS_Validation.csv"))
  colnames(RV_PRS_Validation) <- c("IID","RV_PRS")
  pheno_validation <- inner_join(pheno_validation,RV_PRS_Validation)
  
  model.null <- lm(as.formula(paste0(trait,"~age+age2+sex+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10")),data=pheno_validation)
  pheno_validation$y_validation <- NA
  pheno_validation$y_validation[!is.na(pheno_validation[,trait])] <- model.null$residual
  
  CV_RV_PRS_raw <- pheno_validation
  CV_RV_PRS_adjusted <- pheno_validation
  
  for(i in c("RV_PRS","CV_PRS")){
    tmp <- data.frame(y = CV_RV_PRS_adjusted[,i],CV_RV_PRS_adjusted[,c("PC1","PC2","PC3","PC4","PC5")])
    mod <- lm(y~.,data = tmp)
    R <- mod$residuals
    tmp <- data.frame(y = R^2,CV_RV_PRS_adjusted[,c("PC1","PC2","PC3","PC4","PC5")])
    mod <- lm(y~.,data = tmp)
    y_hat <- predict(mod,tmp)
    if(sum(y_hat < 0) > 0){
      mod <- lm(y~1,data = tmp)
      y_hat <- predict(mod,tmp)
    }
    if(sum(sqrt(y_hat)) == 0){
      CV_RV_PRS_adjusted[,i] <- 0
    }else{
      CV_RV_PRS_adjusted[,i] <- R/sqrt(y_hat)
    }
  }
  
  
  CV_RV_PRS_adjusted$Common_Bin <- 9
  CV_RV_PRS_adjusted$Rare_Bin <- 9
  
  Common_quants <- quantile(CV_RV_PRS_adjusted$CV_PRS,c(0,.1,.2,.3,.4,.6,.7,.8,.9))
  
  for(i in 1:8){
    CV_RV_PRS_adjusted$Common_Bin[CV_RV_PRS_adjusted$CV_PRS < unname(Common_quants[i + 1]) & CV_RV_PRS_adjusted$CV_PRS >= unname(Common_quants[i])] <- i
  }
  
  ukb_pheno <- read.csv("Results/all_phenotypes.csv")
  
  CV_RV_PRS_adjusted_EUR <- CV_RV_PRS_adjusted[CV_RV_PRS_adjusted$IID %in% ukb_pheno$IID[ukb_pheno$ancestry == "EUR"],]
  CV_RV_PRS_adjusted_AMR <- CV_RV_PRS_adjusted[CV_RV_PRS_adjusted$IID %in% ukb_pheno$IID[ukb_pheno$ancestry == "AMR"],]
  CV_RV_PRS_adjusted_AFR <- CV_RV_PRS_adjusted[CV_RV_PRS_adjusted$IID %in% ukb_pheno$IID[ukb_pheno$ancestry == "AFR"],]
  CV_RV_PRS_adjusted_SAS <- CV_RV_PRS_adjusted[CV_RV_PRS_adjusted$IID %in% ukb_pheno$IID[ukb_pheno$ancestry == "SAS"],]
  CV_RV_PRS_adjusted_EAS <- CV_RV_PRS_adjusted[CV_RV_PRS_adjusted$IID %in% ukb_pheno$IID[ukb_pheno$ancestry == "EAS"],]
  CV_RV_PRS_adjusted_MID <- CV_RV_PRS_adjusted[CV_RV_PRS_adjusted$IID %in% ukb_pheno$IID[ukb_pheno$ancestry == "MID"],]
  
  Rare_quants <- quantile(CV_RV_PRS_adjusted_EUR$RV_PRS,c(0,.05,.2,.3,.4,.6,.7,.8,.95))
  for(i in 1:8){
    CV_RV_PRS_adjusted_EUR$Rare_Bin[CV_RV_PRS_adjusted_EUR$RV_PRS < unname(Rare_quants[i + 1]) & CV_RV_PRS_adjusted_EUR$RV_PRS >= unname(Rare_quants[i])] <- i
  }
  
  CV_RV_PRS_adjusted_EUR$Rare_Bin[CV_RV_PRS_adjusted_EUR$Rare_Bin == 1] <- "Below 5%"
  CV_RV_PRS_adjusted_EUR$Rare_Bin[CV_RV_PRS_adjusted_EUR$Rare_Bin %in% c("4","5","6")] <- "30% - 70%"
  CV_RV_PRS_adjusted_EUR$Rare_Bin[CV_RV_PRS_adjusted_EUR$Rare_Bin %in% c("9")] <- "Above 95%"
  
  CV_RV_PRS_adjusted_EUR <- CV_RV_PRS_adjusted_EUR[CV_RV_PRS_adjusted_EUR$Rare_Bin %in% c("Below 5%","30% - 70%","Above 95%"),]
  
  CV_RV_PRS_adjusted_EUR$Rare_Bin <- factor(CV_RV_PRS_adjusted_EUR$Rare_Bin,levels = c("Below 5%","30% - 70%","Above 95%"))
  
  Rare_quants <- quantile(CV_RV_PRS_adjusted_AMR$RV_PRS,c(0,.05,.2,.3,.4,.6,.7,.8,.95))
  for(i in 1:8){
    CV_RV_PRS_adjusted_AMR$Rare_Bin[CV_RV_PRS_adjusted_AMR$RV_PRS < unname(Rare_quants[i + 1]) & CV_RV_PRS_adjusted_AMR$RV_PRS >= unname(Rare_quants[i])] <- i
  }
  
  CV_RV_PRS_adjusted_AMR$Rare_Bin[CV_RV_PRS_adjusted_AMR$Rare_Bin == 1] <- "Below 5%"
  CV_RV_PRS_adjusted_AMR$Rare_Bin[CV_RV_PRS_adjusted_AMR$Rare_Bin %in% c("4","5","6")] <- "30% - 70%"
  CV_RV_PRS_adjusted_AMR$Rare_Bin[CV_RV_PRS_adjusted_AMR$Rare_Bin %in% c("9")] <- "Above 95%"
  
  CV_RV_PRS_adjusted_AMR <- CV_RV_PRS_adjusted_AMR[CV_RV_PRS_adjusted_AMR$Rare_Bin %in% c("Below 5%","30% - 70%","Above 95%"),]
  
  CV_RV_PRS_adjusted_AMR$Rare_Bin <- factor(CV_RV_PRS_adjusted_AMR$Rare_Bin,levels = c("Below 5%","30% - 70%","Above 95%"))
  
  Rare_quants <- quantile(CV_RV_PRS_adjusted_AFR$RV_PRS,c(0,.05,.2,.3,.4,.6,.7,.8,.95))
  for(i in 1:8){
    CV_RV_PRS_adjusted_AFR$Rare_Bin[CV_RV_PRS_adjusted_AFR$RV_PRS < unname(Rare_quants[i + 1]) & CV_RV_PRS_adjusted_AFR$RV_PRS >= unname(Rare_quants[i])] <- i
  }
  
  CV_RV_PRS_adjusted_AFR$Rare_Bin[CV_RV_PRS_adjusted_AFR$Rare_Bin == 1] <- "Below 5%"
  CV_RV_PRS_adjusted_AFR$Rare_Bin[CV_RV_PRS_adjusted_AFR$Rare_Bin %in% c("4","5","6")] <- "30% - 70%"
  CV_RV_PRS_adjusted_AFR$Rare_Bin[CV_RV_PRS_adjusted_AFR$Rare_Bin %in% c("9")] <- "Above 95%"
  
  CV_RV_PRS_adjusted_AFR <- CV_RV_PRS_adjusted_AFR[CV_RV_PRS_adjusted_AFR$Rare_Bin %in% c("Below 5%","30% - 70%","Above 95%"),]
  
  CV_RV_PRS_adjusted_AFR$Rare_Bin <- factor(CV_RV_PRS_adjusted_AFR$Rare_Bin,levels = c("Below 5%","30% - 70%","Above 95%"))
  
  Rare_quants <- quantile(CV_RV_PRS_adjusted_SAS$RV_PRS,c(0,.05,.2,.3,.4,.6,.7,.8,.95))
  for(i in 1:8){
    CV_RV_PRS_adjusted_SAS$Rare_Bin[CV_RV_PRS_adjusted_SAS$RV_PRS < unname(Rare_quants[i + 1]) & CV_RV_PRS_adjusted_SAS$RV_PRS >= unname(Rare_quants[i])] <- i
  }
  
  CV_RV_PRS_adjusted_SAS$Rare_Bin[CV_RV_PRS_adjusted_SAS$Rare_Bin == 1] <- "Below 5%"
  CV_RV_PRS_adjusted_SAS$Rare_Bin[CV_RV_PRS_adjusted_SAS$Rare_Bin %in% c("4","5","6")] <- "30% - 70%"
  CV_RV_PRS_adjusted_SAS$Rare_Bin[CV_RV_PRS_adjusted_SAS$Rare_Bin %in% c("9")] <- "Above 95%"
  
  CV_RV_PRS_adjusted_SAS <- CV_RV_PRS_adjusted_SAS[CV_RV_PRS_adjusted_SAS$Rare_Bin %in% c("Below 5%","30% - 70%","Above 95%"),]
  
  CV_RV_PRS_adjusted_SAS$Rare_Bin <- factor(CV_RV_PRS_adjusted_SAS$Rare_Bin,levels = c("Below 5%","30% - 70%","Above 95%"))
  
  
  Rare_quants <- quantile(CV_RV_PRS_adjusted_EAS$RV_PRS,c(0,.05,.2,.3,.4,.6,.7,.8,.95))
  for(i in 1:8){
    CV_RV_PRS_adjusted_EAS$Rare_Bin[CV_RV_PRS_adjusted_EAS$RV_PRS < unname(Rare_quants[i + 1]) & CV_RV_PRS_adjusted_EAS$RV_PRS >= unname(Rare_quants[i])] <- i
  }
  
  CV_RV_PRS_adjusted_EAS$Rare_Bin[CV_RV_PRS_adjusted_EAS$Rare_Bin == 1] <- "Below 5%"
  CV_RV_PRS_adjusted_EAS$Rare_Bin[CV_RV_PRS_adjusted_EAS$Rare_Bin %in% c("4","5","6")] <- "30% - 70%"
  CV_RV_PRS_adjusted_EAS$Rare_Bin[CV_RV_PRS_adjusted_EAS$Rare_Bin %in% c("9")] <- "Above 95%"
  
  CV_RV_PRS_adjusted_EAS <- CV_RV_PRS_adjusted_EAS[CV_RV_PRS_adjusted_EAS$Rare_Bin %in% c("Below 5%","30% - 70%","Above 95%"),]
  
  CV_RV_PRS_adjusted_EAS$Rare_Bin <- factor(CV_RV_PRS_adjusted_EAS$Rare_Bin,levels = c("Below 5%","30% - 70%","Above 95%"))
  
  
  
  Rare_quants <- quantile(CV_RV_PRS_adjusted_MID$RV_PRS,c(0,.05,.2,.3,.4,.6,.7,.8,.95))
  for(i in 1:8){
    CV_RV_PRS_adjusted_MID$Rare_Bin[CV_RV_PRS_adjusted_MID$RV_PRS < unname(Rare_quants[i + 1]) & CV_RV_PRS_adjusted_MID$RV_PRS >= unname(Rare_quants[i])] <- i
  }
  
  CV_RV_PRS_adjusted_MID$Rare_Bin[CV_RV_PRS_adjusted_MID$Rare_Bin == 1] <- "Below 5%"
  CV_RV_PRS_adjusted_MID$Rare_Bin[CV_RV_PRS_adjusted_MID$Rare_Bin %in% c("4","5","6")] <- "30% - 70%"
  CV_RV_PRS_adjusted_MID$Rare_Bin[CV_RV_PRS_adjusted_MID$Rare_Bin %in% c("9")] <- "Above 95%"
  
  CV_RV_PRS_adjusted_MID <- CV_RV_PRS_adjusted_MID[CV_RV_PRS_adjusted_MID$Rare_Bin %in% c("Below 5%","30% - 70%","Above 95%"),]
  
  CV_RV_PRS_adjusted_MID$Rare_Bin <- factor(CV_RV_PRS_adjusted_MID$Rare_Bin,levels = c("Below 5%","30% - 70%","Above 95%"))
  
  
  
  CV_RV_PRS_adjusted_EUR$y_validation <- scale(CV_RV_PRS_adjusted_EUR$y_validation)
  CV_RV_PRS_adjusted_AMR$y_validation <- scale(CV_RV_PRS_adjusted_AMR$y_validation)
  CV_RV_PRS_adjusted_AFR$y_validation <- scale(CV_RV_PRS_adjusted_AFR$y_validation)
  CV_RV_PRS_adjusted_SAS$y_validation <- scale(CV_RV_PRS_adjusted_SAS$y_validation)
  CV_RV_PRS_adjusted_EAS$y_validation <- scale(CV_RV_PRS_adjusted_EAS$y_validation)
  CV_RV_PRS_adjusted_MID$y_validation <- scale(CV_RV_PRS_adjusted_MID$y_validation)
  
  CV_RV_PRS_adjusted_EUR_se <- aggregate(y_validation ~ Common_Bin + Rare_Bin,data = CV_RV_PRS_adjusted_EUR,function(x){sd(x)/sqrt(length(x))})
  CV_RV_PRS_adjusted_EUR <- aggregate(y_validation ~ Common_Bin + Rare_Bin,data = CV_RV_PRS_adjusted_EUR,mean)
  
  colnames(CV_RV_PRS_adjusted_EUR) <- c("Common_Bin","Rare_Bin","Mean")
  colnames(CV_RV_PRS_adjusted_EUR_se) <- c("Common_Bin","Rare_Bin","SE")
  CV_RV_PRS_adjusted_EUR <- inner_join(CV_RV_PRS_adjusted_EUR,CV_RV_PRS_adjusted_EUR_se)
  
  CV_RV_PRS_adjusted_AMR_se <- aggregate(y_validation ~ Common_Bin + Rare_Bin,data = CV_RV_PRS_adjusted_AMR,function(x){sd(x)/sqrt(length(x))})
  CV_RV_PRS_adjusted_AMR <- aggregate(y_validation ~ Common_Bin + Rare_Bin,data = CV_RV_PRS_adjusted_AMR,mean)
  
  colnames(CV_RV_PRS_adjusted_AMR) <- c("Common_Bin","Rare_Bin","Mean")
  colnames(CV_RV_PRS_adjusted_AMR_se) <- c("Common_Bin","Rare_Bin","SE")
  CV_RV_PRS_adjusted_AMR <- inner_join(CV_RV_PRS_adjusted_AMR,CV_RV_PRS_adjusted_AMR_se)
  
  CV_RV_PRS_adjusted_AFR_se <- aggregate(y_validation ~ Common_Bin + Rare_Bin,data = CV_RV_PRS_adjusted_AFR,function(x){sd(x)/sqrt(length(x))})
  CV_RV_PRS_adjusted_AFR <- aggregate(y_validation ~ Common_Bin + Rare_Bin,data = CV_RV_PRS_adjusted_AFR,mean)
  
  colnames(CV_RV_PRS_adjusted_AFR) <- c("Common_Bin","Rare_Bin","Mean")
  colnames(CV_RV_PRS_adjusted_AFR_se) <- c("Common_Bin","Rare_Bin","SE")
  CV_RV_PRS_adjusted_AFR <- inner_join(CV_RV_PRS_adjusted_AFR,CV_RV_PRS_adjusted_AFR_se)
  
  CV_RV_PRS_adjusted_SAS_se <- aggregate(y_validation ~ Common_Bin + Rare_Bin,data = CV_RV_PRS_adjusted_SAS,function(x){sd(x)/sqrt(length(x))})
  CV_RV_PRS_adjusted_SAS <- aggregate(y_validation ~ Common_Bin + Rare_Bin,data = CV_RV_PRS_adjusted_SAS,mean)
  
  colnames(CV_RV_PRS_adjusted_SAS) <- c("Common_Bin","Rare_Bin","Mean")
  colnames(CV_RV_PRS_adjusted_SAS_se) <- c("Common_Bin","Rare_Bin","SE")
  CV_RV_PRS_adjusted_SAS <- inner_join(CV_RV_PRS_adjusted_SAS,CV_RV_PRS_adjusted_SAS_se)
  
  CV_RV_PRS_adjusted_EAS_se <- aggregate(y_validation ~ Common_Bin + Rare_Bin,data = CV_RV_PRS_adjusted_EAS,function(x){sd(x)/sqrt(length(x))})
  CV_RV_PRS_adjusted_EAS <- aggregate(y_validation ~ Common_Bin + Rare_Bin,data = CV_RV_PRS_adjusted_EAS,mean)
  
  colnames(CV_RV_PRS_adjusted_EAS) <- c("Common_Bin","Rare_Bin","Mean")
  colnames(CV_RV_PRS_adjusted_EAS_se) <- c("Common_Bin","Rare_Bin","SE")
  CV_RV_PRS_adjusted_EAS <- inner_join(CV_RV_PRS_adjusted_EAS,CV_RV_PRS_adjusted_EAS_se)
  
  CV_RV_PRS_adjusted_MID_se <- aggregate(y_validation ~ Common_Bin + Rare_Bin,data = CV_RV_PRS_adjusted_MID,function(x){sd(x)/sqrt(length(x))})
  CV_RV_PRS_adjusted_MID <- aggregate(y_validation ~ Common_Bin + Rare_Bin,data = CV_RV_PRS_adjusted_MID,mean)
  
  colnames(CV_RV_PRS_adjusted_MID) <- c("Common_Bin","Rare_Bin","Mean")
  colnames(CV_RV_PRS_adjusted_MID_se) <- c("Common_Bin","Rare_Bin","SE")
  CV_RV_PRS_adjusted_MID <- inner_join(CV_RV_PRS_adjusted_MID,CV_RV_PRS_adjusted_MID_se)
  
  
  colnames(CV_RV_PRS_adjusted_EUR) <- c("Common_Bin","RICE-RV Quantiles (Rare Variants)","Mean","SE")
  colnames(CV_RV_PRS_adjusted_AMR) <- c("Common_Bin","RICE-RV Quantiles (Rare Variants)","Mean","SE")
  colnames(CV_RV_PRS_adjusted_AFR) <- c("Common_Bin","RICE-RV Quantiles (Rare Variants)","Mean","SE")
  colnames(CV_RV_PRS_adjusted_SAS) <- c("Common_Bin","RICE-RV Quantiles (Rare Variants)","Mean","SE")
  colnames(CV_RV_PRS_adjusted_EAS) <- c("Common_Bin","RICE-RV Quantiles (Rare Variants)","Mean","SE")
  colnames(CV_RV_PRS_adjusted_MID) <- c("Common_Bin","RICE-RV Quantiles (Rare Variants)","Mean","SE")
  
  ymin <- round(min(c(CV_RV_PRS_adjusted_EUR$Mean - CV_RV_PRS_adjusted_EUR$SE,CV_RV_PRS_adjusted_AMR$Mean - CV_RV_PRS_adjusted_AMR$SE,CV_RV_PRS_adjusted_AFR$Mean - CV_RV_PRS_adjusted_AFR$SE,CV_RV_PRS_adjusted_SAS$Mean - CV_RV_PRS_adjusted_SAS$SE,CV_RV_PRS_adjusted_EAS$Mean - CV_RV_PRS_adjusted_EAS$SE,CV_RV_PRS_adjusted_MID$Mean - CV_RV_PRS_adjusted_MID$SE)) - 0.05,2)
  ymax <- round(max(c(CV_RV_PRS_adjusted_EUR$Mean + CV_RV_PRS_adjusted_EUR$SE,CV_RV_PRS_adjusted_AMR$Mean + CV_RV_PRS_adjusted_AMR$SE,CV_RV_PRS_adjusted_AFR$Mean + CV_RV_PRS_adjusted_AFR$SE,CV_RV_PRS_adjusted_SAS$Mean + CV_RV_PRS_adjusted_SAS$SE,CV_RV_PRS_adjusted_EAS$Mean + CV_RV_PRS_adjusted_EAS$SE,CV_RV_PRS_adjusted_MID$Mean + CV_RV_PRS_adjusted_MID$SE)) + 0.05,2)
  
  plot1 <- ggplot(data=CV_RV_PRS_adjusted_EUR, aes(x=Common_Bin, y=Mean, color=`RICE-RV Quantiles (Rare Variants)`)) + geom_line() + geom_pointrange(aes(ymin=Mean-SE, ymax=Mean+SE)) + theme_Publication() + ylab(paste0(ifelse(trait == "logTG","log(TG)",trait)," Standardized")) + ylim(c(ymin,ymax)) +
    scale_x_continuous(breaks = c(1:9),labels = c("0-10%","10-20%","20-30%","30-40%","40-60%","60-70%","70-80%","80-90%","90-100%")) + labs(x = "RICE-CV Quantiles (Common Variants)")
  plot2 <- ggplot(data=CV_RV_PRS_adjusted_AMR, aes(x=Common_Bin, y=Mean, color=`RICE-RV Quantiles (Rare Variants)`)) + geom_line() + geom_pointrange(aes(ymin=Mean-SE, ymax=Mean+SE)) + theme_Publication() + ylab(paste0(ifelse(trait == "logTG","log(TG)",trait)," Standardized")) + ylim(c(ymin,ymax)) +
    scale_x_continuous(breaks = c(1:9),labels = c("0-10%","10-20%","20-30%","30-40%","40-60%","60-70%","70-80%","80-90%","90-100%")) + labs(x = "RICE-CV Quantiles (Common Variants)")
  plot3 <- ggplot(data=CV_RV_PRS_adjusted_AFR, aes(x=Common_Bin, y=Mean, color=`RICE-RV Quantiles (Rare Variants)`)) + geom_line() + geom_pointrange(aes(ymin=Mean-SE, ymax=Mean+SE)) + theme_Publication() + ylab(paste0(ifelse(trait == "logTG","log(TG)",trait)," Standardized")) + ylim(c(ymin,ymax)) +
    scale_x_continuous(breaks = c(1:9),labels = c("0-10%","10-20%","20-30%","30-40%","40-60%","60-70%","70-80%","80-90%","90-100%")) + labs(x = "RICE-CV Quantiles (Common Variants)")
  plot4 <- ggplot(data=CV_RV_PRS_adjusted_SAS, aes(x=Common_Bin, y=Mean, color=`RICE-RV Quantiles (Rare Variants)`)) + geom_line() + geom_pointrange(aes(ymin=Mean-SE, ymax=Mean+SE)) + theme_Publication() + ylab(paste0(ifelse(trait == "logTG","log(TG)",trait)," Standardized")) + ylim(c(ymin,ymax)) +
    scale_x_continuous(breaks = c(1:9),labels = c("0-10%","10-20%","20-30%","30-40%","40-60%","60-70%","70-80%","80-90%","90-100%")) + labs(x = "RICE-CV Quantiles (Common Variants)")
  plot5 <- ggplot(data=CV_RV_PRS_adjusted_EAS, aes(x=Common_Bin, y=Mean, color=`RICE-RV Quantiles (Rare Variants)`)) + geom_line() + geom_pointrange(aes(ymin=Mean-SE, ymax=Mean+SE)) + theme_Publication() + ylab(paste0(ifelse(trait == "logTG","log(TG)",trait)," Standardized")) + ylim(c(ymin,ymax)) +
    scale_x_continuous(breaks = c(1:9),labels = c("0-10%","10-20%","20-30%","30-40%","40-60%","60-70%","70-80%","80-90%","90-100%")) + labs(x = "RICE-CV Quantiles (Common Variants)")
  plot6 <- ggplot(data=CV_RV_PRS_adjusted_MID, aes(x=Common_Bin, y=Mean, color=`RICE-RV Quantiles (Rare Variants)`)) + geom_line() + geom_pointrange(aes(ymin=Mean-SE, ymax=Mean+SE)) + theme_Publication() + ylab(paste0(ifelse(trait == "logTG","log(TG)",trait)," Standardized")) + ylim(c(ymin,ymax)) +
    scale_x_continuous(breaks = c(1:9),labels = c("0-10%","10-20%","20-30%","30-40%","40-60%","60-70%","70-80%","80-90%","90-100%")) + labs(x = "RICE-CV Quantiles (Common Variants)")
  
  
  prow <- plot_grid(
    plot1 + theme(legend.position="none"),
    plot2 + theme(legend.position="none"),
    plot3 + theme(legend.position="none"),
    plot4 + theme(legend.position="none"),
    plot5 + theme(legend.position="none"),
    plot6 + theme(legend.position="none"),
    align = 'vh',
    labels = c("EUR","AMR","AFR","SAS","EAS","MID"),
    hjust = -1,
    ncol = 3, label_size = 20
  )
  
  legend_b <- ggplotGrob(plot1)$grobs[[which(sapply(ggplotGrob(plot1)$grobs, function(x) x$name) == "guide-box")]]
  
  pdf(paste0(trait,"_AoU_WES_Fig5.pdf"), width=24, height=14.8331273177)
  
  print(plot_grid(prow, legend_b, ncol = 1, rel_heights = c(1, .1)))
  
  dev.off()
}

Joining with `by = join_by(IID)`
Joining with `by = join_by(IID)`
Joining with `by = join_by(Common_Bin, Rare_Bin)`
Joining with `by = join_by(Common_Bin, Rare_Bin)`
Joining with `by = join_by(Common_Bin, Rare_Bin)`
Joining with `by = join_by(Common_Bin, Rare_Bin)`
Joining with `by = join_by(Common_Bin, Rare_Bin)`
Joining with `by = join_by(Common_Bin, Rare_Bin)`
Warning message:
“Removed 4 rows containing missing values or values outside the scale range
(`geom_segment()`).”
Joining with `by = join_by(IID)`
Joining with `by = join_by(IID)`
Joining with `by = join_by(Common_Bin, Rare_Bin)`
Joining with `by = join_by(Common_Bin, Rare_Bin)`
Joining with `by = join_by(Common_Bin, Rare_Bin)`
Joining with `by = join_by(Common_Bin, Rare_Bin)`
Joining with `by = join_by(Common_Bin, Rare_Bin)`
Joining with `by = join_by(Common_Bin, Rare_Bin)`
Warning message:
“Removed 1 row containing missing values or values outside the scale range
(`geom_segment()`).”
Warning message:
“Removed 6 rows containi

In [3]:
system("rm -r Results")

In [4]:
system("mkdir Results")

In [5]:
system("gsutil cp gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Train.txt Results/")

In [6]:
for(anc in c("AMR")){
    for(trait in c("HDL")){
        system(paste0("gsutil cp gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/GWAS_SummaryStats/",anc,"_",trait,"_GWAS_SumStats_Cleaned_QQPlots Results/"))
    }
}

In [7]:
list.files("Results")

[1] "All_Train.txt"                        
[2] "AMR_HDL_GWAS_SumStats_Cleaned_QQPlots"

In [8]:
install.packages("plotrix")

Installing package into ‘/home/jupyter/packages’
(as ‘lib’ is unspecified)



In [9]:
install.packages("ggthemes")

Installing package into ‘/home/jupyter/packages’
(as ‘lib’ is unspecified)



In [10]:
rm(list = ls())

pheno_train <- read.delim("Results/All_Train.txt")

theme_Publication <- function(base_size=12) {
  library(grid)
  library(ggthemes)
  (theme_foundation(base_size=base_size, )
    + theme(plot.title = element_text(face = "bold",
                                      size = rel(1.1), hjust = 0.5),
            text = element_text(),
            panel.background = element_rect(colour = NA),
            plot.background = element_rect(colour = NA),
            panel.border = element_rect(colour = NA),
            axis.title = element_text(face = "bold",size = 16),
            axis.title.y = element_text(angle=90,vjust =2),
            axis.title.x = element_blank(),
            axis.text.x = element_blank(), 
            axis.line = element_line(colour="black",size=2),
            axis.ticks = element_line(),
            title = element_text(face = "bold",size = 20),
            # panel.grid.major = element_line(colour="#f0f0f0"),
            # panel.grid.minor = element_line(colour="#f0f0f0"),
            panel.grid.major = element_blank(),
            panel.grid.minor = element_blank(),
            legend.key = element_rect(colour = NA),
            #legend.position = "bottom",
            #legend.direction = "horizontal",
            #legend.key.size= unit(0.2, "cm"),
            #legend.margin = unit(0, "cm"),
            legend.title = element_text(face="bold.italic", size =18),
            #legend.text = element_text(face ="bold"),
            plot.margin=unit(c(10,5,5,5),"mm"),
            strip.background=element_rect(colour="#f0f0f0",fill="#f0f0f0"),
            strip.text = element_text(face="bold")
    ))
  
}

library(plotrix)
library(data.table)
library(RColorBrewer)
library(dplyr)
library(ggplot2)

lambda_dat <- NULL

for(anc in c("AMR")){
    for(trait in c("HDL")){
  dat <- fread(paste0("Results/",anc,"_",trait,"_GWAS_SumStats_Cleaned_QQPlots"))
  dat <- as.data.frame(dat)
  colnames(dat) <- c("CHR","SNP","BP","A1_FREQ","P")
  dat$MAF <- ifelse(dat$A1_FREQ <= 0.5, dat$A1_FREQ,1-dat$A1_FREQ)
  dat <- dat[!is.na(dat$MAF),]
  dat <- dat[!is.na(dat$P),]
  dat <- dat[dat$MAF > 0.01,]
  dat <- dat[dat$P != 0,]
        
if(anc == "AFR"){
    anc_plot <- "African"
}else if(anc == "AMR"){
    anc_plot <- "Latino"
}else{
    anc_plot <- "European"
}
  
  
  x <- dat$P
  z <- qnorm(x / 2)
  lambda <- round(median(z^2) / qchisq(0.5,1), 3)
  lambda_1000 <- round(1+1000*(lambda-1)/sum(!is.na(pheno_train[,trait])) ,3)
  
  lambda_dat <- rbind(lambda_dat,data.frame(Trait = trait,Ancestry_Group = anc_plot,lambda = lambda, lambda_1000 = lambda_1000, datasource = "AoU WES"))
  
  p.pwas <- 5E-08
  
  nCHR <- length(unique(dat$CHR))
  dat$BPcum <- NA
  s <- 0
  nbp <- c()
  for (i in unique(dat$CHR)){
    nbp[i] <- max(dat[dat$CHR == i,]$BP)
    dat$BPcum[dat$CHR == i] <- dat$BP[dat$CHR == i] + s
    s <- s + nbp[i]
  }
  axis.set <- dat %>% 
    group_by(CHR) %>% 
    summarize(center = (max(BPcum) + min(BPcum)) / 2)
  ylim <- abs(floor(log10(min(dat$P)))) + 2 
  sig1 <- p.pwas
  
  sigline <- data.frame(sig=c(-log10(sig1)),val=c(paste0("P=",signif(sig1,2))))
  
  p1 <- ggplot(dat, aes(x = BPcum, y = -log10(P), 
                        color = as.factor(CHR), size = -log10(P))) +
    geom_point(alpha = 0.8, size=0.8) + 
    scale_x_continuous(label = axis.set$CHR, breaks = axis.set$center) +
    scale_y_continuous(expand = c(0,0), limits = c(0, ylim)) +
    scale_color_manual(values = rep(c("#08306b", "#4292c6"), nCHR)) +
    scale_size_continuous(range = c(0.5,3)) +
    geom_hline(data = sigline, aes(yintercept = sig), color= "red", linetype="dashed") +
    guides(color = F) + 
    labs(x = NULL, 
         y = "-log10(p)", 
         linetype = "",
         title = paste0(trait," (",anc,")"))+
    theme_Publication()+
    theme(
      legend.position = "top",
      panel.border = element_blank(),
      panel.grid.major.x = element_blank(),
      panel.grid.minor.x = element_blank(),
      axis.text.x = element_text(angle = 0, size = 16, vjust = 0.5),
      axis.text.y = element_text(size = 16),
      plot.subtitle = element_text(size = 8)
    )
  
  ggsave(paste0(trait,"_",anc,"_AoU_WES_CV_Manhattan_Plot.png"),p1,width = 15,height = 9.27070457355,dpi = 300)
  
  png(paste0(trait,"_",anc,"_AoU_WES_CV_QQplot.png"), width=10, height=10,units = "in",res = 300)
  
  qqplotdata <- function(logpvector){
    o = sort(logpvector,decreasing=T)
    e = -log10(ppoints(length(o)))       
    qqdata <- data.frame(o,e)
    qqdata$o <- round(qqdata$o,3)
    qqdata$e <- round(qqdata$e,3)
    keepU <- which(!duplicated(qqdata))
    qqdata <- qqdata[keepU,]
    
    N <- length(logpvector) ## number of p-values
    ## create the confidence intervals
    qqdata$c975 <- NA
    qqdata$c025 <- NA
    
    ## the jth order statistic from a
    ## uniform(0,1) sample
    ## has a beta(j,n-j+1) distribution
    ## (Casella & Berger, 2002,
    ## 2nd edition, pg 230, Duxbury)
    
    for(i in 1:length(keepU)){
      j <- keepU[i]
      qqdata$c975[i] <- -log10(qbeta(0.975,j,N-j+1))
      qqdata$c025[i] <- -log10(qbeta(0.025,j,N-j+1))
    }
    return(qqdata)
  }
  yLine <- c(-log10(5E-8))
  colLine <- c("red")
  dat$log10P <- -log10(dat$P)
  gwas <- as.data.frame(dat)
  # Determine frequency bins and create variable for binned QQ plot
  
  minMAF <- min(gwas$MAF)
  
  freqbins <- c(c(0.5,0.05,0.01,0)[which(c(0.5,0.05,0.01,0) > floor(minMAF*1000000)/1000000)],floor(minMAF*1000000)/1000000)
  gwas$freqbin <- cut(gwas$MAF, freqbins,include.lowest=T)
  freqtable <- table(gwas$freqbin)
  freqtable <- freqtable[order(-as.numeric(gsub("[\\[\\(](.+),.+","\\1",names(freqtable))))]
  freqtable <- freqtable[freqtable > 0]
  
  ## Generate QQ plot data by frequency bin
  fbin <- character(0)
  fN <- integer(0)
  fx <- numeric(0)
  fy <- numeric(0)
  fcol <- character(0)
  legendcol <- character(0)
  conf <- list()
  allcols <- brewer.pal(4,"Set1")
  ycol <- "log10P"
  for(f in 1:length(freqtable)){
    fbin <- c(fbin,names(freqtable)[f])
    fsnps <- which(gwas$freqbin ==names(freqtable)[f])
    plotdata <- qqplotdata(gwas[[ycol]][fsnps])
    fN <- c(fN,freqtable[f])
    fx <- c(fx,plotdata$e)
    fy <- c(fy,plotdata$o)
    fcol <- c(fcol,rep(allcols[f],length(plotdata$o)))
    conf[[f]] <- data.frame('x'=c(plotdata$e,rev(plotdata$e)),
                            'y'=c(plotdata$c975,rev(plotdata$c025)))
    legendcol <- c(legendcol,allcols[f])
  }
  legendtext <- paste0("MAF=",fbin,"; N SNPs=",format(fN,big.mark=",",scientific=FALSE))
  opt <-  list(break.top = 15,
               top.size = 0.125)
  
  
  xlim <- c(0,max(fx,na.rm=T))
  ylim <- c(0,max(fy,na.rm=T))
  maxY <- max(fy,na.rm=T)
  print("okkkk2")
  par(mar=c(5.1,5.1,4.1,1.1))
  print("okkkk3")
  
  lab1 <- pretty(c(0,opt$break.top),n=ceiling(12 * (1-opt$top.size)))
  lab1 <- c(lab1[lab1 < opt$break.top],opt$break.top)
  lab2 <- pretty(c(opt$break.top,maxY),n=max(3,floor(12 * opt$top.size)))
  lab2 <- lab2[lab2 > max(lab1)]
  
  # resulting range of top scale in bottom scale units
  top.range = opt$break.top/(1 - opt$top.size) - opt$break.top
  top.data = max(lab2)-opt$break.top
  
  # function to rescale the top part
  rescale <- function(y) { opt$break.top+(y-opt$break.top)/(top.data/top.range)}
  rescaled.y <- rescale(fy[fy>opt$break.top])
  plot(0,0,
       ylim=c(min(fy),opt$break.top*(1+opt$top.size)),xlim=xlim,axes=FALSE,
       xlab=expression(plain(Expected)~~group("(",-log[10]*italic(P),")")),
       ylab=expression(plain(Observed)~~group("(",-log[10]*italic(P),")")),
       cex=1,cex.lab=1.5,cex.axis=1.5,bty="n",col="transparent",
       main=opt$maintitle,pch=19)
  
  # Plot confidence intervals	
  for(p in 1:length(conf)){
    polygon(conf[[p]]$'x',ifelse(conf[[p]]$'y'>opt$break.top,rescale(conf[[p]]$'y'),conf[[p]]$'y'),
            col=grDevices::rgb(t(grDevices::col2rgb(allcols[p])),alpha=50,maxColorValue=255),
            border = NA)
  }
  
  # add points
  points(fx[fy<opt$break.top],fy[fy<opt$break.top],cex=1,col=fcol[fy<opt$break.top],pch=19)
  
  # identify line & add axis break
  lines(xlim,xlim,col="black",lty = 2)
  axis(1,cex.axis=1.5,cex.lab=1.5)
  par(las=1)
  axis(side=2,at=lab1,cex.axis=1.5,cex.lab=1.5)
  par(las=0)
  box()
  par(las=0)
  points(fx[fy>opt$break.top],rescaled.y,cex=1,col=fcol[fy>opt$break.top],pch=19)
  par(las=1)
  axis(side=2,at=rescale(lab2),labels=lab2,cex.axis=1.5,cex.lab=1.5)
  axis.break(axis=2,breakpos=opt$break.top,style="zigzag",brw=0.02)
  axis.break(axis=4,breakpos=opt$break.top,style="zigzag",brw=0.02)
  lines(range(fx),c(opt$break.top,opt$break.top),col = "grey",lty = 6)
  abline(h=ifelse(yLine<opt$break.top,
                  yLine,
                  rescale(yLine)),
         col=colLine,lwd=1.5,lty=2)
  legend("topleft",legend=legendtext,col=legendcol,pch=15,bty="n",cex=1.5)
  text(4,1,expression(paste(lambda[1000]," = ")),cex = 1.5)
  text(4.65,1,paste(lambda_1000),cex = 1.5)
  title(paste0(trait," (",anc,")"),cex.main = 1.5)
  
  # p2 <- recordPlot()
  # 
  # title <- ggdraw() + 
  #   draw_label(
  #     paste0(trait," for Europeans"),
  #     fontface = 'bold',
  #     x = 0,
  #     hjust = 0,
  #     size = 30
  #   ) +
  #   theme(
  #     # add margin on the left of the drawing canvas,
  #     # so title is aligned with left edge of first plot
  #     plot.margin = margin(0, 0, 0, 7)
  #   )
  
  # plot_row <- plot_grid(p1, p2, align = "h", axis = "bt", rel_widths = c(6, 1))
  # 
  # p3 <- plot_grid(
  #   title, plot_row,
  #   ncol = 1,
  #   # rel_heights values control vertical title margins
  #   rel_heights = c(0.1, 1)
  # )
  # 
  # cowplot::save_plot(filename = paste0(trait,"_UKB_WES_CV_QQplot.pdf"),plot = p3,nrow = 2,ncol = 2,base_width = c(24))
  # 
  dev.off()
}
}
write.csv(lambda_dat,file = "AoU_WES_lambda.csv",row.names = FALSE)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Warning message:
“The `<scale>` argument of `guides()` cannot be `FALSE`. Use "none" instead as
of ggplot2 3.3.4.”
Warning message:
“The `size` argument of `element_line()` is deprecated as of ggplot2 3.4.0.
ℹ Please use the `linewidth` argument instead.”


[1] "okkkk2"
[1] "okkkk3"
